In [5]:
#!/usr/bin/python
def calculate(clf, dataset, feature_list, folds = 1000):
    from tester import StratifiedShuffleSplit 
    data = featureFormat(dataset, feature_list, sort_keys = True)
    labels, features = targetFeatureSplit(data)
    cv = StratifiedShuffleSplit(n_splits=folds, random_state=42)
    true_negatives = 0
    false_negatives = 0
    true_positives = 0
    false_positives = 0
    for train_idx, test_idx in cv.split(features, labels):
        features_train = []
        features_test  = []
        labels_train   = []
        labels_test    = []
        for ii in train_idx:
            features_train.append( features[ii] )
            labels_train.append( labels[ii] )
        for jj in test_idx:
            features_test.append( features[jj] )
            labels_test.append( labels[jj] )
        
        ### fit the classifier using training set, and test on test set
        clf.fit(features_train, labels_train)
        predictions = clf.predict(features_test)
        for prediction, truth in zip(predictions, labels_test):
            if prediction == 0 and truth == 0:
                true_negatives += 1
            elif prediction == 0 and truth == 1:
                false_negatives += 1
            elif prediction == 1 and truth == 0:
                false_positives += 1
            elif prediction == 1 and truth == 1:
                true_positives += 1
            else:
                print("Warning: Found a predicted label not == 0 or 1.")
                print("All predictions should take value 0 or 1.")
                print("Evaluating performance for processed predictions:")
                break
    try:
        total_predictions = true_negatives + false_negatives + false_positives + true_positives
        accuracy = 1.0*(true_positives + true_negatives)/total_predictions
        precision = 1.0*true_positives/(true_positives+false_positives)
        recall = 1.0*true_positives/(true_positives+false_negatives)
        f1 = 2.0 * true_positives/(2*true_positives + false_positives+false_negatives)
        f2 = (1+2.0*2.0) * precision*recall/(4*precision + recall)
        accuracy = round(accuracy,2)
        precision= round(precision,2)
        recall= round(recall,2)
        
        return accuracy, precision, recall
#         print(clf)
#         print(PERF_FORMAT_STRING.format(accuracy, precision, recall, f1, f2, display_precision = 5))
#         print(RESULTS_FORMAT_STRING.format(total_predictions, true_positives, false_positives, false_negatives, true_negatives))
#         print("")
    except:
        print("Got a divide by zero when trying out:", clf)
        print("Precision or recall may be undefined due to a lack of true positive predicitons.")
        
import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list = ['poi', 'bonus', 'expenses', 
                   'other', 
                  'restricted_stock',
                 ]
### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "rb") as data_file:
    data_dict = pickle.load(data_file)

### Task 2: Remove outliers
from remove_outlier import removeOutlier
data_dict = removeOutlier(data_dict)
### Task 3: Create new feature(s)
### Store to my_dataset for easy export below.
my_dataset = data_dict
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

def reDraw(leaf,split):
    from sklearn.tree import DecisionTreeClassifier
    clf = DecisionTreeClassifier(min_samples_leaf=leaf,min_samples_split=split)
    accuracy, precision, recall = calculate(clf, my_dataset, features_list)
    Label(root, text=accuracy).grid(row=3, column=1)
    Label(root, text=precision).grid(row=4, column=1)
    Label(root, text=recall).grid(row=5, column=1)

def getInputs():
    try: 
        leaf = int(tolNentry.get())
    except:
        leaf = 1
#         print "enter Integer for tolN"
        tolNentry.delete(0, END)
        tolNentry.insert(0,'1')
    try: 
        split = int(tolSentry.get())
    except:
        split = 2
#         print "enter Float for tolS"
        tolSentry.delete(0, END)
        tolSentry.insert(0,'2')
    return leaf,split

def drawNewTree():
    leaf,split = getInputs()

    
    reDraw(leaf,split)
    
    
from numpy import *
from tkinter import *
# import regTrees
root=Tk()

Label(root, text="decision tree").grid(row=0,columnspan=3)
Label(root, text="min_samples_leaf").grid(row=1, column=0)
tolNentry = Entry(root)
tolNentry.grid(row=1, column=1)
tolNentry.insert(0,'1')
Label(root, text="min_samples_split>=2").grid(row=2, column=0)
tolSentry = Entry(root)
tolSentry.grid(row=2, column=1)
tolSentry.insert(0,'2')
Button(root, text="ReDraw", command=drawNewTree).grid(row=1, column=2,\
                                                 rowspan=3)


Label(root, text="Accuracy:").grid(row=3, column=0)
Label(root, text="Precision:").grid(row=4, column=0)
Label(root, text="Recall:").grid(row=5, column=0)


root.mainloop() 